In [89]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
data=pd.read_csv('work.csv')
data1=data.copy()
from sklearn.feature_selection import VarianceThreshold
select=VarianceThreshold(threshold=0.02)
x=data1.drop(['TARGET'],axis=1)
y=data1['TARGET']
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
x,y = ros.fit_resample(x, y)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [90]:
select.fit(x_train)
x_train=select.transform(x_train)
x_test=select.transform(x_test)
x_train=pd.DataFrame(x_train)
x_test=pd.DataFrame(x_test)
duplicated=x_train.T.duplicated()
columns=[not index for index in duplicated]
x_train=(x_train.T[columns]).T
x_test=(x_test.T[columns]).T

In [91]:
# x_train=np.array(x_train).reshape(116819, 241,1)
# x_train=np.array(x_test).reshape(29205, 241,1)
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(x_train)
x_train=scaler.transform(x_train)
x_test=scaler.transform(x_test)
x_train=np.array(x_train).reshape(116819, 241,1)
x_test=np.array(x_test).reshape(29205, 241,1)

In [110]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout,Dense,Flatten,Embedding,Activation,BatchNormalization
from tensorflow.keras.layers import Conv1D,MaxPooling1D,GlobalMaxPooling1D
model=Sequential()
model.add(Conv1D(32,3,activation='relu',input_shape=(241,1)))
model.add(BatchNormalization())
model.add(MaxPooling1D(2))
model.add(Dropout(0.5))

model.add(Conv1D(64,3,activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(2))
model.add(Dropout(0.5))

model.add(Conv1D(128,3,activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(2))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

In [117]:
# # columns=x_train[x_train.columns[select.get_support(indices=True)]].columns
# columns=[0, 2, 145, 160, 167, 285, 286, 287, 288, 324]
# # 2, 23, 56, 76, 78, 119, 126, 136, 163, 219
# # 164, 165, 167, 168, 169, 171, 172, 173, 175, 179

model.compile(optimizer='adam',
              loss='binary_crossentropy', 
             metrics=[tf.keras.metrics.AUC()])
import numpy
y_train=y_train.reshape(116819,-1)
y_test=y_test.reshape(29205,-1)



In [118]:

# important=pd.concat([pd.Series(analysis[0]),pd.Series(analysis[1])],axis=1)
# important.columns=['anova','p_value']
# important.sort_values(by='anova',ascending=False,inplace=True)
# important.iloc[:10,:]
model.fit(x_train,y_train,epochs=10,batch_size=1024, validation_data=(x_test,y_test))

Train on 116819 samples, validate on 29205 samples
Epoch 1/10
116819/116819 [==============================] - 234s 2ms/sample - loss: 0.6632 - auc_19: 0.7067 - val_loss: 0.9887 - val_auc_19: 0.7577
Epoch 2/10
116819/116819 [==============================] - 243s 2ms/sample - loss: 0.5773 - auc_19: 0.7747 - val_loss: 1.1720 - val_auc_19: 0.7462
Epoch 3/10
116819/116819 [==============================] - 245s 2ms/sample - loss: 0.5598 - auc_19: 0.7897 - val_loss: 0.9504 - val_auc_19: 0.7622
Epoch 4/10
116819/116819 [==============================] - 248s 2ms/sample - loss: 0.5526 - auc_19: 0.7962 - val_loss: 0.5735 - val_auc_19: 0.8111
Epoch 5/10
116819/116819 [==============================] - 249s 2ms/sample - loss: 0.5481 - auc_19: 0.8002 - val_loss: 0.5408 - val_auc_19: 0.8169
Epoch 6/10
116819/116819 [==============================] - 244s 2ms/sample - loss: 0.5426 - auc_19: 0.8045 - val_loss: 0.5320 - val_auc_19: 0.8211
Epoch 7/10
116819/116819 [==============================] - 2

In [129]:
# x_train=x_train[columns]
# x_test=x_test[columns]
y_pred=model.predict(x_train1)

In [132]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import confusion_matrix
# from sklearn.feature_selection import SelectFromModel
# sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))
# sel.fit(x_train, y_train)
# # clf = RandomForestClassifier(n_estimators=100, random_state=0)
# # y_pred=clf.predict(x_train1)
# selected_feat= x_train.columns[(sel.get_support())]
for i in range(len(y_pred)):
    if(y_pred[i]>0.5):
        y_pred[i]=1
    else:
        y_pred[i]=0
  

In [23]:
# columns=[  0,   2,   5,  12,  14,  15,  17,  18,  23,  56,  78, 114, 118,
#             122, 131, 141, 145, 160, 167, 169, 208, 237, 238, 239, 240, 241,
#             242, 243, 252, 281, 282, 283, 284, 285, 286, 287, 288, 324]

In [1]:
# submission = pd.DataFrame({"TARGET":y_pred})
# # submission.to_csv("submission.csv", index=False)

(116819, 38)

In [36]:
# from sklearn.model_selection import cross_validate
# clf=RandomForestClassifier(n_estimators=100)
# cross_validate(clf, x_train, y_train, cv=10, scoring='roc_auc')

{'fit_time': array([37.9361639 , 36.481215  , 36.62443042, 37.11823082, 36.70355415,
        36.45906448, 37.19196963, 37.46967506, 37.02883363, 36.77787852]),
 'score_time': array([0.87125087, 0.7063024 , 0.69315457, 0.69231343, 0.70011878,
        0.56154227, 0.70516562, 0.71807241, 0.69913578, 0.70575833]),
 'test_score': array([0.99991462, 0.99991465, 0.99991451, 0.99991457, 0.99999997,
        0.99991462, 0.99991454, 1.        , 0.9997435 , 0.99991463])}

In [17]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
# data=pd.read_csv('work.csv')
# data.shape


(76020, 371)

In [128]:
data=pd.read_csv('test.csv')
data1=data.copy()
x_train1=data1
x_train1=select.transform(x_train1)
x_train1=pd.DataFrame(x_train1)
x_train1=(x_train1.T[columns]).T
x_train1=scaler.transform(x_train1)
x_train1=np.array(x_train1).reshape(75818, 241,1)
x_train1.shape

(75818, 241, 1)

In [136]:
data1=pd.read_csv('sample_submission.csv')
data1['TARGET'].value_counts()

0    71516
1     4302
Name: TARGET, dtype: int64

In [137]:
data1['TARGET']=y_pred
data=pd.concat([data1['ID'],data1['TARGET']],axis=1)

In [138]:
data.to_csv('submission.csv',index=False)

In [139]:
data['TARGET'].value_counts()

0.0    51849
1.0    23969
Name: TARGET, dtype: int64

In [140]:
data1=pd.read_csv('submission.csv')
data1['TARGET'].value_counts()

0.0    51849
1.0    23969
Name: TARGET, dtype: int64

In [123]:
x_train.shape

(116819, 241, 1)

0.705281446